In [1]:
from surface_roughness import Surface
import numpy as np

surface = Surface(path='example_surface.stl')

x:\python_env\roughness_3_8\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
v0 = surface.points[surface.triangles[:,0]]
v1 = surface.points[surface.triangles[:,1]]
v2 = surface.points[surface.triangles[:,2]]

v1v0 = v1 - v0
v2v0 = v2 - v0

normals = np.cross(v1v0,v2v0,axisa=1,axisb=1)

areas = np.linalg.norm(normals,axis=1)
print(areas)
normals /= areas[:,np.newaxis]
print(normals)

[0.04934573 0.04729415 0.022432   ... 0.06028062 0.06245704 0.0446357 ]
[[ 0.06985742  0.06810488  0.99522945]
 [ 0.16007916  0.13612797  0.97767266]
 [ 0.19769169  0.10317618  0.9748193 ]
 ...
 [ 0.2379187   0.08708326  0.96737335]
 [ 0.21033365  0.10238662  0.97225343]
 [ 0.14691056 -0.10905404  0.98311978]]


In [3]:
shear_dir = np.array([1,0,0])

proj = np.zeros_like(normals)
proj[:,0] = normals[:,0] - (shear_dir[1]**2*normals[:,0] - shear_dir[0]*shear_dir[1]*normals[:,1])
proj[:,1] = normals[:,1] - (shear_dir[0]*shear_dir[1]*normals[:,0] + shear_dir[0]**2 * normals[:,1])
proj[:,2] = normals[:,2]

proj /= np.linalg.norm(proj,axis=1)[:,np.newaxis]
app_dip = np.arccos(proj[:,0]*shear_dir[0]+proj[:,1]*shear_dir[1])-np.pi/2
print(app_dip)

[-0.07007734 -0.16229482 -0.20008477 ... -0.24115668 -0.21305301
 -0.1483354 ]


In [4]:
facing_areas = areas[app_dip>0]
facing_appdips = np.degrees(app_dip[app_dip>0])

delta_t = np.sum(facing_appdips * facing_areas) / facing_areas.sum()
delta_star_t = np.sqrt(np.sum(facing_areas * facing_appdips**2)/facing_areas.sum())
print(len(facing_areas))
print(delta_t)
print(delta_star_t)

7984
7.282562247623986
9.195512678145327
